<a href="https://colab.research.google.com/github/Nivedha1205/E-Commerce-Customer-analytics-platform/blob/main/E_commerce_Customer_Analytics_platform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
path = "/content/Online Retail.xlsx"

In [3]:
sales_data = pd.read_excel(path)

In [4]:
sales_data.shape

(541909, 8)

In [5]:
sales_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
sales_data["CustomerID"].isna()

,CustomerID
0,False
1,False
2,False
3,False
4,False
...,...
541904,False
541905,False
541906,False
541907,False


In [7]:
sales_data = sales_data.dropna(subset=["CustomerID"])
sales_data = sales_data.reset_index(drop=True)

In [8]:
sales_data.shape

(406829, 8)

In [9]:
sales_data = sales_data[~sales_data['InvoiceNo'].astype(str).str.startswith('C')]
sales_data.shape

(397924, 8)

In [10]:
sales_data['Total_price'] = sales_data['UnitPrice'] * sales_data['Quantity']
sales_data.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total_price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [11]:
sales_data['CustomerID'].nunique()

4339

In [12]:
sales_data['Description'].nunique()

3877

In [13]:
sales_data['Country'].nunique()

37

In [14]:
sales_data['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Finland',
       'Austria', 'Greece', 'Singapore', 'Lebanon',
       'United Arab Emirates', 'Israel', 'Saudi Arabia', 'Czech Republic',
       'Canada', 'Unspecified', 'Brazil', 'USA', 'European Community',
       'Bahrain', 'Malta', 'RSA'], dtype=object)

In [15]:
sales_data['InvoiceNo'].nunique()

18536

In [16]:
sales_data['Total_price'].sum()

np.float64(8911407.904)

In [17]:
sales_data['InvoiceDate'] = pd.to_datetime(sales_data['InvoiceDate'])
last_date = sales_data['InvoiceDate'].max()
print('last_purchase_date:', last_date)

last_purchase_date: 2011-12-09 12:50:00


In [18]:
reference_date = last_date + pd.Timedelta(days=1)
print('reference_date:', reference_date)

reference_date: 2011-12-10 12:50:00


In [19]:
rfm = sales_data.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,
    'InvoiceNo': 'count',
    'Total_price': 'sum'
})
rfm.head()

,InvoiceDate,InvoiceNo,Total_price
CustomerID,,,
12346.0,326,1,77183.60
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40


In [20]:
rfm.rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'Total_price': 'Monetary'
}, inplace=True)
print(rfm.head())

            Recency  Frequency  Monetary
CustomerID                              
12346.0         326          1  77183.60
12347.0           2        182   4310.00
12348.0          75         31   1797.24
12349.0          19         73   1757.55
12350.0         310         17    334.40


In [21]:
rfm['r_rank'] = rfm['Recency'].rank(method='first', ascending=True)
rfm['f_rank'] = rfm['Frequency'].rank(method='first', ascending=False)
rfm['m_rank'] = rfm['Monetary'].rank(method='first', ascending=False)

In [22]:
rfm['R_score'] = pd.cut(rfm['r_rank'], bins=5, labels=[5,4,3,2,1])
rfm['F_score'] = pd.cut(rfm['f_rank'], bins=5, labels=[1,2,3,4,5])
rfm['M_score'] = pd.cut(rfm['m_rank'], bins=5, labels=[1,2,3,4,5])

In [23]:
rfm['RFM_Score'] = rfm['R_score'].astype(str) + rfm['F_score'].astype(str) + rfm['M_score'].astype(str)

In [24]:
print(rfm.head())

            Recency  Frequency  Monetary  r_rank  f_rank  m_rank R_score  \
CustomerID                                                                 
12346.0         326          1  77183.60  4176.0  4268.0    10.0       1   
12347.0           2        182   4310.00    94.0   515.0   336.0       5   
12348.0          75         31   1797.24  2676.0  2489.0  1006.0       2   
12349.0          19         73   1757.55  1127.0  1444.0  1034.0       4   
12350.0         310         17    334.40  4107.0  3231.0  3110.0       1   

           F_score M_score RFM_Score  
CustomerID                            
12346.0          5       1       151  
12347.0          1       1       511  
12348.0          3       2       232  
12349.0          2       2       422  
12350.0          4       4       144  
